In [60]:
# Setup
if __name__ == '__main__':
    import os
    # Change the current working directory to the parent directory of this file
    os.chdir(os.path.dirname(os.path.dirname(os.path.dirname(__vsc_ipynb_file__))))

from evaluation import get_actual_demand
from system_state import SystemState
from utils import load_problem_data
import numpy as np
import pandas as pd
import uuid
import json

demand, datacenters, servers, selling_prices = load_problem_data()
system_state = SystemState(datacenters, servers)

seed = 4507
np.random.seed(seed)
actual_demand = get_actual_demand(demand)

In [61]:
from greedy_profit_v2.data import get_sorted_servers, break_even_time_all
from greedy_profit_v2.results import save_results_as_actions

def remaining_slots_decrement_algorithm(latency: str, server: str, remaining_slots: pd.DataFrame, desired_buy_count: int, time_step_range: tuple[int, int], use_dc4=False):
    results = []

    if 'CPU' in server:
        slots_size = 2
    else:
        slots_size = 4

    if latency == 'low':
        datacenter_id = 'DC1'
    elif latency == 'medium':
        datacenter_id = 'DC2'
    elif latency == 'high':
        if use_dc4:
            datacenter_id = 'DC4'
        else:
            datacenter_id = 'DC3'
    
    while True:
        # 1) Find the ranges of time steps between which the datacenter can fit at least 1 server
        relevant_remaining_slots = remaining_slots.loc[time_step_range[0]:time_step_range[1]].query(f"{datacenter_id} >= @slots_size")
        time_steps_with_space = relevant_remaining_slots.index.to_numpy()

        if relevant_remaining_slots.empty:
            break

        time_steps_diff = np.diff(time_steps_with_space)
        gap_indices = np.append(np.where(time_steps_diff > 1), len(time_steps_with_space) - 1)

        ranges = []
        start = 0
        for gap in gap_indices:
            ranges.append((time_steps_with_space[start], time_steps_with_space[gap]))
            start = gap + 1
        # print(ranges)


        # 2) Filter ranges which last for less than the break even time
        break_even_time = break_even_time_all[server][latency]
        ranges = [r for r in ranges if r[1] - r[0] >= (2*break_even_time)]


        # 3) For each range (from longest to shortest):
        sorted_ranges_i = np.argsort([r[1] - r[0] for r in ranges])
        for i in reversed(sorted_ranges_i):
            current_range = ranges[i]

            # 1) Calculate the minimum slot capacity
            remaining_slots_in_range = relevant_remaining_slots.loc[current_range[0]:current_range[1]]
            # print(remaining_slots_in_range)
            minimum_slot_capacity = remaining_slots_in_range[datacenter_id].min()


            # 2) Pick the minimum of the minimum slot capacity or the number of servers to buy * server slots size
            max_buy_count = int(np.floor(minimum_slot_capacity / slots_size))
            actual_buy_count = min(max_buy_count, desired_buy_count)
            # print(f"{actual_buy_count} servers out of {desired_buy_count} are being purchased...")

            if actual_buy_count < desired_buy_count:
                print(f"{actual_buy_count} servers out of {desired_buy_count} bought, {datacenter_id} is full at {relevant_remaining_slots.query(f"{datacenter_id} == @minimum_slot_capacity").index}")


            # 3) Store the number of servers to buy, the datacentre, the buy time step, the dismiss time step in the results
            results.append({
                    'server_generation': server,
                    'buy_count': str(actual_buy_count),
                    'datacenter_id': datacenter_id,
                    'buy_time_step': str(current_range[0]),
                    'dismiss_time_step': str(current_range[1] + 1)
                })


            # 4) Subtract the number of bought servers from the initial desired number of servers to buy
            desired_buy_count -= actual_buy_count


            # 5) For each slot capacity in the range for the datacenter, decrease the slot capacity by the result of step 3.2
            actual_slot_count = actual_buy_count * slots_size
            # print(f"Subtracting {actual_slot_count} ({actual_buy_count} servers) from the remaining slots in the {datacenter_id} ({current_range[0]}:{current_range[1]})")
            for index, row in remaining_slots_in_range.iterrows():
                remaining = row[datacenter_id] - actual_slot_count
                remaining_slots.at[index, datacenter_id] = remaining
        


        # 4) Repeat steps 1 to 3.3 until there are no ranges left after step 2
        if len(ranges) == 0 or desired_buy_count <= 0:
            # print("No more ranges of demand to satisfy")
            # results_df = pd.DataFrame(results)
            # total_servers_bought = results_df['buy_count'].astype(int).sum()
            # print(f"Total servers bought: {total_servers_bought}")
            break

    # 5) Return a tuple of the (results, remaining_slots)
    return results, remaining_slots, desired_buy_count

    

In [77]:
# Simulate the algorithm with the most profitable server/latency


results = []
sorted_servers = get_sorted_servers('data/test_data/most_profitable_servers_by_artem.csv')

# 1) Initialise a DataFrame that tracks the remaining slots of each datacentre at each time step
datacenter_slot_capacities = { 'DC1': 25245, 'DC2': 15300, 'DC3': 7020, 'DC4': 8280 }
remaining_slots = pd.DataFrame(datacenter_slot_capacities, index=range(1, 169), columns=['DC1', 'DC2', 'DC3', 'DC4']).rename_axis('time_step')
# print(remaining_slots)

# 2) For each server/latency combination (in order of profitability):
for server_generation, latency_sensitivity in sorted_servers:
    remaining_demand = actual_demand.copy()


    while True:
        # 1) Find the ranges of time steps between which this server/latency is in demand
        relevant_demand = remaining_demand.query(f'server_generation == @server_generation and {latency_sensitivity} > 0')
        # print(relevant_demand)
        time_steps_of_demand = relevant_demand.get('time_step').to_numpy()
        # print(time_steps_of_demand)

        time_steps_diff = np.diff(time_steps_of_demand)
        gap_indices = np.append(np.where(time_steps_diff > 1), len(time_steps_of_demand) - 1)

        ranges = []
        start = 0
        for gap in gap_indices:
            ranges.append((time_steps_of_demand[start], time_steps_of_demand[gap]))
            start = gap + 1

        # print(ranges)

        # 2) Merge ranges which have a negligibly small gap in between (relative to the length of the smallest range)
        i = 0
        while i < len(ranges) - 1:
            length = ranges[i][1] - ranges[i][0]
            length_next = ranges[i + 1][1] - ranges[i + 1][0]
            if ranges[i + 1][0] - ranges[i][1] < (min(length, length_next)/4 * 17):
                ranges[i] = (ranges[i][0], ranges[i + 1][1])
                ranges.pop(i + 1)
            else:
                i += 1
        # print(ranges)

        # 3) Filter all ranges which last for less than the time it takes for the server/latency to break even
        break_even_time = break_even_time_all[server_generation][latency_sensitivity]
        ranges = [r for r in ranges if r[1] - r[0] >= (2*break_even_time)]

        # print(ranges)

        
        # 4) Truncate all ranges that are too large (>96) by continuously removing the lesser end until the range is <=96 TODO NOT WORTH IT
        # for i in range(0, len(ranges)):
        #     head = ranges[i][0]
        #     tail = ranges[i][1]
        #     too_long = head - tail > 96
        #     while head - tail > 96:
        #         head_demand = relevant_demand.set_index('time_step').loc[head][latency_sensitivity]
        #         tail_demand = relevant_demand.set_index('time_step').loc[tail][latency_sensitivity]
        #         if head_demand < tail_demand:
        #             head += 1
        #         else:
        #             tail += 1
        #     if too_long:
        #         ranges[i] = (head, tail)
                

        
        # 3) For each range (from longest to shortest):
        sorted_ranges_i = np.argsort([r[1] - r[0] for r in ranges])
        for i in reversed(sorted_ranges_i):
            # i = 2
            current_range = ranges[i]
            # print(range)

            # 1) Calculate the minimum demand across that range
            demand_in_range = relevant_demand.query(f'time_step >= @current_range[0] and time_step <= @current_range[1]')
            min_demand = demand_in_range.min()[latency_sensitivity]
            # 2) Calculate the number of servers to buy meet the minimum demand
            capacity = servers.set_index('server_generation').loc[server_generation]['capacity']
            desired_buy_count = int(np.round(min_demand / capacity))
            # print(f"{min_demand}/{capacity} = {min_demand / capacity} ~~ {str(desired_buy_count)} GPUs to buy")

            # 3) Perform the Remaining Slot Decrement Algorithm
            validated_results, remaining_slots, leftover_buy_count = remaining_slots_decrement_algorithm(latency_sensitivity, server_generation, remaining_slots, desired_buy_count, current_range)
            results.extend(validated_results)

            # 3.1) If there are leftover serveers to buy and the latency is high, perform the Remaining Slot Decrement Algorithm with DC4
            if latency_sensitivity == 'high' and leftover_buy_count > 0:
                dc4_results, remaining_slots, dc4_leftover_buy_count = remaining_slots_decrement_algorithm(latency_sensitivity, server_generation, remaining_slots, leftover_buy_count, current_range, use_dc4=True)
                print(f"{current_range}: {leftover_buy_count} should be bought in DC4")
                results.extend(dc4_results)

            # 5) For each demand in the range, subtract the capacity * number of servers to buy
            demand_to_subtract = desired_buy_count * capacity
            # print(f"Subtracting {demand_to_subtract} from the demand in the range")
            for index, row in demand_in_range.iterrows():
                remaining = row[latency_sensitivity] - demand_to_subtract
                remaining_demand.at[index, latency_sensitivity] = remaining


            # 6) Filter new demand values which are too low to buy at least 1 server for
            remaining_demand = remaining_demand.query(f'{latency_sensitivity} > {(capacity / 2) + 1}')


            
        # 4) Repeat steps 1.1 to 1.3.4 with the new demand values until there are no ranges after 1.2
        if len(ranges) == 0:
            # print("No more ranges of demand to satisfy")
            results_df = pd.DataFrame(results)
            total_servers_bought = results_df['buy_count'].astype(int).sum()
            print(f"{server_generation}, {latency_sensitivity}: Total servers bought: {total_servers_bought}")
            break


remaining_slots.to_csv('./remaining_slots.csv', sep='\t', encoding='utf-8')
# save_json('./results.json', results)
save_results_as_actions('./4507_too_large_truncate.json', results)

GPU.S3, high: Total servers bought: 550
GPU.S3, medium: Total servers bought: 753
GPU.S3, low: Total servers bought: 796
GPU.S2, high: Total servers bought: 885
GPU.S1, high: Total servers bought: 1560
GPU.S2, medium: Total servers bought: 1665
GPU.S1, medium: Total servers bought: 1746
GPU.S2, low: Total servers bought: 1921
GPU.S1, low: Total servers bought: 2030
CPU.S4, high: Total servers bought: 4343
59 servers out of 65 bought, DC3 is full at Index([156], dtype='int64', name='time_step')
15 servers out of 24 bought, DC3 is full at Index([154, 155], dtype='int64', name='time_step')
1 servers out of 43 bought, DC3 is full at Index([153], dtype='int64', name='time_step')
52 servers out of 59 bought, DC3 is full at Index([152], dtype='int64', name='time_step')
19 servers out of 34 bought, DC3 is full at Index([143, 144], dtype='int64', name='time_step')
(132, 168): 5 should be bought in DC4
(132, 168): 2 should be bought in DC4
(132, 168): 18 should be bought in DC4
(132, 168): 2 sho